In [2]:
from datasets import load_dataset

ds = load_dataset("MikhailT/lj-speech")

README.md:   0%|          | 0.00/456 [00:00<?, ?B/s]

(…)-00000-of-00008-0cc4e31b6b21f574.parquet:   0%|          | 0.00/479M [00:00<?, ?B/s]

(…)-00001-of-00008-3b5b422725ba5106.parquet:   0%|          | 0.00/477M [00:00<?, ?B/s]

(…)-00002-of-00008-a5f87edb1313613a.parquet:   0%|          | 0.00/474M [00:00<?, ?B/s]

(…)-00003-of-00008-f2519bfc406b1727.parquet:   0%|          | 0.00/467M [00:00<?, ?B/s]

(…)-00004-of-00008-40ebdc0ae3e773f7.parquet:   0%|          | 0.00/465M [00:00<?, ?B/s]

(…)-00005-of-00008-8040cd75cc070bf2.parquet:   0%|          | 0.00/458M [00:00<?, ?B/s]

(…)-00006-of-00008-cd1094b775fb9f17.parquet:   0%|          | 0.00/487M [00:00<?, ?B/s]

(…)-00007-of-00008-cdc74d3724524931.parquet:   0%|          | 0.00/478M [00:00<?, ?B/s]

Generating full split:   0%|          | 0/13100 [00:00<?, ? examples/s]

In [1]:
from hydra import initialize, compose
from omegaconf import OmegaConf

# configs/ 가 있는 디렉토리 기준
with initialize(config_path="../configs", version_base="1.3"):
    # train.yaml의 defaults를 해석하고, 필요한 오버라이드를 한 번에 적용
    cfg = compose(
        config_name="train.yaml",
        overrides=[
            "data=ljspeech",
            "model=matcha",
            "trainer=default",
            # 예: "trainer.max_epochs=50" 같은 개별 오버라이드도 가능
        ],
    )

ocfg = OmegaConf.to_yaml(cfg)

In [2]:
from omegaconf import OmegaConf
import hydra

model = hydra.utils.instantiate(cfg.model)

/usr/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/diffusers/models/lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)


In [3]:
cfg.data.train_filelist_path = '/workspace/matchatrain/LJSpeech-1.1/train.csv'
cfg.data.valid_filelist_path = '/workspace/matchatrain/LJSpeech-1.1/test.csv'

datamodule = hydra.utils.instantiate(cfg.data)

In [4]:
cfg.data

{'_target_': 'matcha.data.text_mel_datamodule.TextMelDataModule', 'name': 'ljspeech', 'train_filelist_path': '/workspace/matchatrain/LJSpeech-1.1/train.csv', 'valid_filelist_path': '/workspace/matchatrain/LJSpeech-1.1/test.csv', 'batch_size': 32, 'num_workers': 20, 'pin_memory': True, 'cleaners': ['english_cleaners2'], 'add_blank': True, 'n_spks': 1, 'n_fft': 1024, 'n_feats': 80, 'sample_rate': 22050, 'hop_length': 256, 'win_length': 1024, 'f_min': 0, 'f_max': 8000, 'data_statistics': {'mel_mean': -5.536622, 'mel_std': 2.116101}, 'seed': '${seed}', 'load_durations': False}

In [5]:
datamodule.setup()

In [10]:
data=next(iter(datamodule.train_dataloader()))

In [27]:
from matcha.data.text_mel_datamodule import TextMelDataModule, parse_filelist

TextMelDataModule(
    name='ljspeech',
    train_filelist_path=cfg.data.train_filelist_path,
    valid_filelist_path=cfg.data.valid_filelist_path,
    batch_size=cfg.data.batch_size,
    num_workers=cfg.data.num_workers,
    pin_memory=cfg.data.pin_memory,
    cleaners=cfg.data.cleaners,
    add_blank=cfg.data.add_blank,
    n_spks=cfg.data.n_spks,
    n_fft=cfg.data.n_fft,
    n_feats=cfg.data.n_feats,
    sample_rate=cfg.data.sample_rate,
    hop_length=cfg.data.hop_length,
    win_length=cfg.data.win_length,
    f_min=cfg.data.f_min,
    f_max=cfg.data.f_max,
    data_statistics=cfg.data.data_statistics, # mel_mean= -5.536622, mel_std= 2.116101
    seed=1234,
    load_durations=False
)

In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("/workspace/matchatrain/LJSpeech-1.1/metadata.csv", sep="|", header=None)
# fillist = [[df.iloc[i][0], df.iloc[i][1]] for i in range(len(df))]

# train(95%), test(5%) 랜덤 분할
train_df, test_df = train_test_split(df, test_size=0.05, random_state=42)

# 각각 저장
train_df.to_csv("train.csv", sep="|", index=False, header=False)
test_df.to_csv("test.csv", sep="|", index=False, header=False)

In [46]:
from tqdm import tqdm

for d in tqdm(fillist):
    if not os.path.exists('/workspace/matchatrain/LJSpeech-1.1/wavs/' + d[0] + '.wav'):
        print(d)

100%|██████████| 13100/13100 [00:16<00:00, 799.93it/s] 


In [34]:
print(df.iloc[0][1])
print(df.iloc[0][2])

Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition
Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition


In [36]:
import os

os.path.exists('/workspace/matchatrain/LJSpeech-1.1/wavs/LJ001-0001.wav')

LJ001-0001


True